In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import LabelEncoder
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, make_scorer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from collections import Counter
import nltk

In [2]:
train = pd.read_csv(r'C:\Users\lizac\Downloads\h-1b-visa\h1b_kaggle.csv')
#test = pd.read_csv(r'C:\Users\lizac\Downloads\h-1b-visa\h1b_kaggle.csv')

In [3]:
df=pd.DataFrame(train)
#df=df.head(10000)

In [ ]:
train.groupby('CASE_STATUS').size()

In [724]:
df = df.drop(df[(df.CASE_STATUS =='WITHDRAWN') | (df.CASE_STATUS=='PENDING QUALITY AND COMPLIANCE REVIEW - UNASSIGNED' )|
                (df.CASE_STATUS =='REJECTED') | (df.CASE_STATUS =='INVALIDATED')| (df.CASE_STATUS.isnull()) |(df.JOB_TITLE.isnull())
                | (df.FULL_TIME_POSITION.isnull())].index)
dict_map = {'CERTIFIED':1 , 'CERTIFIED-WITHDRAWN': 1, 'DENIED': 0}
df['CASE_STATUS'] = df['CASE_STATUS'].map(dict_map)
df['CASE_STATUS'] = df['CASE_STATUS'].astype(int)
dict_map = {'N':0 , 'Y': 1}
df['FULL_TIME_POSITION'] = df['FULL_TIME_POSITION'].map(dict_map)
df['FULL_TIME_POSITION'] = df['FULL_TIME_POSITION'].astype(int)

In [4]:
train.groupby('CASE_STATUS').size()

CASE_STATUS
CERTIFIED                                             2615623
CERTIFIED-WITHDRAWN                                    202659
DENIED                                                  94346
INVALIDATED                                                 1
PENDING QUALITY AND COMPLIANCE REVIEW - UNASSIGNED         15
REJECTED                                                    2
WITHDRAWN                                               89799
dtype: int64

In [673]:
def a(text):
    if 'PRESIDEN' in text:
        text='PRESIDENT'
    return text

In [674]:
df['JOB_TITLE']=df['JOB_TITLE'].map(lambda x: a(x))

In [675]:
stops = set(stopwords.words("english"))
def cleanData(text,remove_stops = False, stemming = False):
    txt = str(text)
    txt = re.sub(r'[^A-Za-z0-9\s]',r'',txt)
    txt = re.sub(r'\n',r' ',txt)
    
    
        
    if remove_stops:
        txt = " ".join([w for w in txt.split() if w not in stops])
    return txt

In [676]:

df['JOB_TITLE']=df['JOB_TITLE'].map(lambda x: cleanData(x, remove_stops=True, stemming=True))

In [677]:
df['EMPLOYER_NAME']=df['EMPLOYER_NAME'].map(lambda x: cleanData(x, remove_stops=True, stemming=True))

In [678]:
text=pd.DataFrame(df['JOB_TITLE'])

In [679]:
text=pd.DataFrame(text)

In [680]:
text=text.to_string(index=False)

In [681]:
splitter = re.compile('[^a-zA-Z0-9_\\+\\-/]')
words = []
for single_word in splitter.split(text):
        current_word = single_word.strip().lower()
        if len(current_word) > 3 and current_word != '':
            words.append(current_word)


In [682]:
nouns = []
for word,pos in nltk.pos_tag(nltk.word_tokenize(str(text))):
         if (pos == 'NN' or pos == 'NNP' or pos == 'NNS' or pos == 'NNPS'):
            nouns.append(word)

In [683]:
professions=[]
for word in nouns:
         if word.endswith(('ST', 'ER','OR','LOW','DENT','CEO','HEAD','EXECUTIVE')) :
            professions.append(word)
     

In [684]:
common = Counter(professions).most_common()

In [685]:
common=pd.DataFrame(common)

In [686]:
common=common[0].values

In [687]:
common=common.tolist()

In [688]:
#common=common.remove('JUNIOR')
#common=common.remove('SENIOR')

In [689]:

#train.JOB_TITLE = train.JOB_TITLE.astype(str)

In [690]:
#df['tokenized_text'] = df.JOB_TITLE.apply(lambda row: word_tokenize(row))

In [691]:
#text=df['JOB_TITLE']
#text=text.head(200)
#text=pd.DataFrame(text)
#text = text.astype(str)

In [692]:
df['tokenized_text'] = df['JOB_TITLE'].apply(lambda row: word_tokenize(row))

In [693]:
def new_prof(text):
    new1=[]
    for index in range(0,len(text)):
        if text[index] in common:
            new1=text[index]
    return new1

In [694]:
df['new_title'] = df.tokenized_text.apply(lambda row: new_prof(row))

In [695]:
df['JOB_TITLE']=df['new_title']

In [696]:
#df['new_title'][290]==[]

In [697]:
df['tokenized_worksite'] = df['WORKSITE'].apply(lambda row: row.split(','))

In [698]:
df['WORKSITE'] = df['tokenized_worksite'].apply(lambda row: row[1])

In [699]:
list_drop=['YEAR','lon','lat','tokenized_text','new_title','tokenized_worksite']

In [700]:
df.drop(list_drop, axis=1, inplace=True)

In [701]:
job_title= df.JOB_TITLE.astype(str).str.strip('[]').str.get_dummies()#
job_title.columns = job_title.columns.str.strip("")

In [702]:
soc_name= df.SOC_NAME.astype(str).str.strip('[]').str.get_dummies()
soc_name.columns = soc_name.columns.str.strip("")

In [703]:
empl_name= df.EMPLOYER_NAME.astype(str).str.strip('[]').str.get_dummies()
empl_name.columns = empl_name.columns.str.strip("")

In [704]:

worksite= df.WORKSITE.astype(str).str.strip('[]').str.get_dummies()
worksite.columns = worksite.columns.str.strip("")

In [705]:
df=pd.concat([df,job_title,soc_name,empl_name,worksite],axis=1)

In [706]:
list_drop=['JOB_TITLE','SOC_NAME','EMPLOYER_NAME','WORKSITE','Unnamed: 0','FULL_TIME_POSITION','PREVAILING_WAGE']

In [707]:
df.drop(list_drop, axis=1, inplace=True)

In [708]:
y=df['CASE_STATUS']


In [709]:
x=df.iloc[0:,1:]

In [710]:
from sklearn.model_selection import train_test_split

In [711]:
Xtrain, Xtest, ytrain, ytest = train_test_split(x, y, random_state=2)
print(Xtrain.shape, Xtest.shape)

(7185, 5677) (2395, 5677)


In [712]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import roc_auc_score, classification_report

In [713]:
model_l = LogisticRegression(random_state = 1)

In [714]:
model_l.fit(Xtrain, ytrain)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=1, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [715]:
model_l.score(Xtest,ytest)

0.9586638830897704

In [716]:
model_l.score(Xtrain,ytrain)

0.9585247042449547